In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

# ---------------------------
# Functions for the Pipeline
# ---------------------------

def parse_c_file(file_path):
    """
    Parse the OpenFOAM cell centers file (named "C") to extract (x,y) coordinates.
    Expects the file to contain an 'internalField' entry.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_idx = None
    num_centres = None
    for i, line in enumerate(lines):
        if "internalField" in line:
            num_centres = int(lines[i+1].strip())
            start_idx = i + 3
            break

    if start_idx is None or num_centres is None:
        raise ValueError("Could not find 'internalField' or the number of cell centres in the file")

    coordinates = []
    for line in lines[start_idx:start_idx+num_centres]:
        if "(" in line and ")" in line:
            x, y, _ = map(float, line.strip("()\n").split())
            coordinates.append([x, y])
    return np.array(coordinates)


def reshape_trajectory_data(sim_data, cell_centers, grid_shape):
    """
    Reshape simulation data (timesteps, num_cells, 4) to a fixed grid of shape
    (timesteps, n_rows, n_cols, 5). The five channels are:
       - Channel 0: ρ
       - Channel 1: Ux
       - Channel 2: Uy
       - Channel 3: P
       - Channel 4: hole indicator (0 if cell exists; 1 if hole)
    
    Each cell center (x,y) is mapped to a grid index based on the bounding box of cell_centers:
    
         col = round((x - x_min) / (x_max - x_min) * (n_cols - 1))
         row = round((y - y_min) / (y_max - y_min) * (n_rows - 1))
    
    Cells that are not filled by any simulation cell remain zero in the first four channels,
    and their hole indicator is left as 1.
    """
    n_rows, n_cols = grid_shape
    T = sim_data.shape[0]
    
    # Determine domain boundaries from cell centers
    x_min, x_max = np.min(cell_centers[:, 0]), np.max(cell_centers[:, 0])
    y_min, y_max = np.min(cell_centers[:, 1]), np.max(cell_centers[:, 1])
    
    # Initialize grid with zeros; add one extra channel for the binary mask
    reshaped = np.zeros((T, n_rows, n_cols, 5))
    # Initialize mask grid to 1 (indicating hole everywhere)
    mask = np.ones((n_rows, n_cols))
    
    # Precompute mapping from each simulation cell to grid indices
    mapping = []
    for (x, y) in cell_centers:
        col = int(round((x - x_min) / (x_max - x_min) * (n_cols - 1)))
        row = int(round((y - y_min) / (y_max - y_min) * (n_rows - 1)))
        mapping.append((row, col))
        mask[row, col] = 0  # mark that a cell exists here

    # For each timestep, fill in the simulation data and the binary mask
    for t in range(T):
        for i, (row, col) in enumerate(mapping):
            reshaped[t, row, col, 0:4] = sim_data[t, i, :]  # assign the 4 simulation channels
            reshaped[t, row, col, 4] = 0  # cell exists → mask 0
        # For positions that were not filled, the mask remains 1.
        reshaped[t, :, :, 4] = mask

    return reshaped


def combine_and_reshape_trajectories(dataset, cell_centers, grid_shape):
    """
    Combine all trajectory data (from a single results.npy file with shape
         (num_trajectories, timesteps, num_cells, 4))
    and reshape them into a fixed grid of shape
         (num_trajectories, timesteps, n_rows, n_cols, 5)
    using the provided cell centers and grid shape.
    """
    num_trajectories = dataset.shape[0]
    combined_list = []
    
    for i in tqdm(range(num_trajectories), desc="Reshaping Trajectories"):
        # Extract the simulation data for trajectory i (all timesteps, all cells, all 4 channels)
        sim_data = dataset[i, :, :, :]  # shape: (timesteps, 16320, 4)
        reshaped_data = reshape_trajectory_data(sim_data, cell_centers, grid_shape)  # shape: (timesteps, 128, 128, 5)
        combined_list.append(reshaped_data)
        gc.collect()  # free memory if needed
        
    combined = np.array(combined_list)
    return combined


def plot_reshaped_sample(trajectory, timestep, output_folder, channel_names=["ρ", "Ux", "Uy", "P", "mask"]):
    """
    Plot the 5 channels of a single timestep from a reshaped trajectory.
    
    Args:
        trajectory (np.ndarray): Array of shape (timesteps, 128, 128, 5).
        timestep (int): Time index to plot.
        output_folder (str): Directory to save the plot.
        channel_names (list): Names for the 5 channels.
    """
    data = trajectory[timestep]  # shape: (128, 128, 5)
    n_channels = data.shape[-1]
    
    fig, axes = plt.subplots(1, n_channels, figsize=(4*n_channels, 4))
    for ch in range(n_channels):
        im = axes[ch].imshow(data[:, :, ch], cmap="viridis")
        axes[ch].set_title(f"{channel_names[ch]} | Timestep {timestep}")
        axes[ch].axis("off")
        fig.colorbar(im, ax=axes[ch], fraction=0.046, pad=0.04)
    plt.tight_layout()
    
    plot_path = os.path.join(output_folder, f"trajectory_sample_t{timestep}.png")
    plt.savefig(plot_path)
    plt.close(fig)
    print(f"Sample plot saved to {plot_path}")

# ---------------------------
# Main Pipeline
# ---------------------------
def main():
    # Define file paths (update these paths as needed)
    data_path = "/home/openfoamuser/Geo-UPSplus/dataset_gen/Flowbench_Openfoam/FPO_cylinder/Irregular/results.npy"  # Shape: (600,20,16320,4)
    c_file_path = "/home/openfoamuser/Geo-UPSplus/dataset_gen/Flowbench_Openfoam/FPO_cylinder/Irregular/Design_Point_0_copy_1/0/C"  # OpenFOAM C file
    output_folder = "output"
    os.makedirs(output_folder, exist_ok=True)
    
    # Load simulation data
    print(f"Loading simulation data from {data_path}")
    data = np.load(data_path)  # Expected shape: (600,20,16320,4)
    print(f"Data shape: {data.shape}")
    
    # Parse cell centers from the C file
    print(f"Parsing cell centers from C file: {c_file_path}")
    cell_centers = parse_c_file(c_file_path)
    print(f"Cell centers shape: {cell_centers.shape}")
    
    # Define target grid shape (128 x 128)
    grid_shape = (128, 128)
    
    # Combine and reshape all trajectories
    combined_data = combine_and_reshape_trajectories(data, cell_centers, grid_shape)
    print(f"Combined reshaped data shape: {combined_data.shape}")  # Expected: (600,20,128,128,5)
    
    # Save the combined reshaped data for later use
    combined_save_path = os.path.join(output_folder, "combined_reshaped.npy")
    np.save(combined_save_path, combined_data)
    print(f"Combined reshaped data saved to {combined_save_path}")
    
    # Plot a sample trajectory for visual inspection (e.g., first trajectory, timestep 0)
    sample_trajectory = combined_data[0]  # shape: (20,128,128,5)
    sample_timestep = 0
    plot_reshaped_sample(sample_trajectory, sample_timestep, output_folder)
    
if __name__ == "__main__":
    main()


Loading simulation data from /home/openfoamuser/Geo-UPSplus/dataset_gen/Flowbench_Openfoam/FPO_cylinder/Irregular/results.npy
Data shape: (1, 20, 16320, 4)
Parsing cell centers from C file: /home/openfoamuser/Geo-UPSplus/dataset_gen/Flowbench_Openfoam/FPO_cylinder/Irregular/Design_Point_0_copy_1/0/C
Cell centers shape: (16320, 2)


Reshaping Trajectories:   0%|          | 0/1 [00:00<?, ?it/s]

Reshaping Trajectories: 100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


Combined reshaped data shape: (1, 20, 128, 128, 5)
Combined reshaped data saved to output/combined_reshaped.npy
Sample plot saved to output/trajectory_sample_t0.png
